# Цель работы

**Цель** - автоматизировать базовую отчетность нашего приложения. Наладить автоматическую отправку аналитической сводки в телеграм каждое утро в 11.00.

**Идея** - загрузить данные с Clickhouse, выбрать основные метрики и собрать отчет, который в последствии будет приходит в чате в телеграмме каждое утро.

**Задачи**:

1. Возьмем основные метрики - DAU, Просмотры, Лайки, CTR.
2. Напишем функцию для создания сообщения с информацией о значениях ключевых метрик за предыдущий день.
3. Затем функцию для графиков - отобразим последние 7 дней, чтобы лучше было видно динамику.
4. Отправляем данные в чат.

*Сам скрипт DAGа в конце*

# Автоматизация отчетов

## Загрузка необходимых элементов для бота

In [229]:
# Загружаем необходимые библиотеки
from datetime import datetime, timedelta
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import pandas as pd
import requests

from airflow.decorators import dag, task
from airflow.operators.python import get_current_context
import pandahouse


# Устанавливаем свзять с ClickHouse
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20230320'
}

# Получаем доступ к боту
my_token = '6245347830:AAHFIuYvAh9YxKR4taSMoQHNA0GuelpvUrA' # токен
bot = telegram.Bot(token=my_token)

# Получаем необходимый ID
chat_id = -802518328

In [ ]:
# А чтоюы получить необходимый ID
updates = bot.getUpdates()
chat_id = 1013581629

## Отчет по ленте новостей

### Ключевые метрики (за предыдущий день)

In [232]:
def analytics_last_day():
    '''Предоставляет данные за прошлый день о:
        - DAU
        - CTR
        - Просмотры
        - Лайки'''
    # Берем данные с ClikHouse (за предыдущий день)
    query = '''
                SELECT
                    toDate(time) as date,
                    count(DISTINCT(user_id)) as DAU,
                    sum(action = 'like') as likes,
                    sum(action = 'view') as views,
                    likes / views as CTR
                FROM simulator_20230320.feed_actions 
                WHERE toDate(time) = yesterday()
                GROUP BY date
            '''
    data = pandahouse.read_clickhouse(query, connection=connection)

    # Сохраняем дату
    date = data.date[0].strftime('%d.%m.%Y')
    msg_date = f'*Аналитика по ленте новостей за {date}*'

    ## Cоздаем сообщения
    # DAU
    msg_DAU = f'DAU: {data.DAU[0]}'
    # CTR
    msg_CTR = f'CTR: {round(data.CTR[0], 2)}'
    # Просмторы
    msg_views = f'Просмотры: {data.views[0]}'
    # Лайки
    msg_likes = f'Лайки: {data.likes[0]}'
    # Объединяем
    msg = msg_date + '\n\n' + \
          msg_DAU + '\n' + \
          msg_CTR + '\n' + \
          msg_views + '\n' + \
          msg_likes

    return ms

### Графики со значениями метрик (за предыдущие 7 дней)

In [233]:
def graphs_last_7_day():
    '''Предоставляет данные за прошлый день о:
        - DAU
        - CTR
        - Просмотры
        - Лайки'''
    # Берем данные с ClikHouse (за предыдущие 7 дней)
    query_7 = '''
                SELECT 
                    toStartOfHour(time) as date,
                    count(DISTINCT(user_id)) as users_count,
                    sum(action = 'like') as likes,
                    sum(action = 'view') as views,
                    likes / views as CTR
                FROM simulator_20230320.feed_actions 
                WHERE (toDate(date) > today() - 7) AND (toDate(date) != today())
                GROUP BY date
              '''
    data_7 = pandahouse.read_clickhouse(query_7, connection=connection)

    # Создаем настройки для объединения графиков
    fig, ax = plt.subplots(2, 2, figsize=(19, 8), sharex=True)
    fig.suptitle(f'Аналитика за за предыдущие 7 дней', fontweight='bold')
    sns.set_style("whitegrid")
        
    # Задаем позиции
    ax_1 = ax[0, 0]
    ax_2 = ax[0, 1]
    ax_3 = ax[1, 0]
    ax_4 = ax[1, 1]

    ## DAU
    # Создаем график
    sns.lineplot(data=data_7, x='date', y='users_count', ax=ax_1)
    # Добавляем подписи
    ax_1.set_title(f'DAU за последние 7 дней', fontsize=12)
    ax_1.set_xlabel('Даты')
    ax_1.set_ylabel('Кол-во уник. польз.')

    ## CTR
    # Создаем график
    sns.lineplot(data=data_7, x='date', y='CTR',ax=ax_2)
    # Добавляем подписи
    ax_2.set_title(f'CTR за последние 7 дней', fontsize=12)
    ax_2.set_xlabel('Даты')
    ax_2.set_ylabel('CTR')

    ## Просмотры
    # Создаем график
    sns.lineplot(data=data_7, x='date', y='views', ax=ax_3)
    # Добавляем подписи
    ax_3.set_title(f'Число просмотров за последние 7 дней', fontsize=12)
    ax_3.set_xlabel('Даты')
    ax_3.set_ylabel('Число просмотров')

    ## Лайки
    # Создаем график
    sns.lineplot(data=data_7, x='date', y='likes', ax=ax_4)
    # Добавляем подписи
    ax_4.set_title(f'Число лайков за последние 7 дней', fontsize=12)
    ax_4.set_xlabel('Даты')
    ax_4.set_ylabel('Число лайков')

    # Сохраняем в буфер
    plot_object = io.BytesIO()
    plt.savefig(plot_object)
    plot_object.seek(0)
    plot_object.name = f'Аналитика.png'
    plt.close()

    return plot_object

## Отправка в чат

In [234]:
def send_analytics(msg, plot_object):
    # Отправляем в чат сообщение
    bot.sendMessage(chat_id=chat_id, text=msg, parse_mode = "Markdown")
    # Отправляем график в чат
    bot.sendPhoto(chat_id=chat_id, photo=plot_object)

## Запуск

In [235]:
msg = analytics_last_day()
plot_object = graphs_last_7_day()
send_analytics(msg, plot_object)

# Код Дага